<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out = "".join(
        [
            booster_version
            for i, booster_version in enumerate(table_cells.strings)
            if i % 2 == 0
        ][0:-1]
    )
    return out


def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out = [i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0 : mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = " ".join(row.contents)

    # Filter the digit and empty names
    if not (colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/91.0.4472.124 Safari/537.36"
}

## Data collection process (technical summary)

This section documents the exact steps taken to collect and produce the web-scraped CSV used in subsequent analysis. It is written for an engineer who will reproduce or adapt the pipeline.

1. Snapshot selection and reproducibility

   - We scrape a static snapshot of the Wikipedia page by including the page `oldid` parameter in the URL (`static_url`). This ensures the HTML structure and content remain stable across runs and makes the extraction reproducible.
   - Using a static URL avoids drift when the live page changes; include the `oldid` value in any future reproducibility notes or commits.

2. HTTP request settings and polite scraping

   - A custom `User-Agent` header is supplied in `headers` to identify the client and avoid basic bot-blocking. In production, add delays (sleep) between requests and respect `robots.txt` when scraping multiple pages.
   - For single-page extraction we perform a single GET; if you expand to pagination, implement rate-limiting and exponential backoff on 429/5xx responses.

3. HTML parsing and robust element selection

   - The response HTML is parsed with BeautifulSoup (`html.parser`). We locate the target table(s) by tag/class (here the third `wikitable plainrowheaders collapsible` table).
   - We avoid brittle position-only selectors where possible; prefer finding tables by distinctive class attributes and then validating column headers before parsing rows.

4. Header extraction and normalization

   - Table header `<th>` cells are cleaned with `extract_column_from_header()` to remove nested tags (`<a>`, `<sup>`, `<br>`) and whitespace, producing canonical column names.
   - Any duplicate or irrelevant header (e.g., an empty date/time header) is explicitly removed and replaced with a consistent set of keys used for the output dictionary.

5. Row parsing with helper functions

   - For each table row, helper functions (`date_time`, `booster_version`, `landing_status`, `get_mass`, `_get_first_link_or_text`) are used to reliably extract and normalize cell contents: split date/time, normalize booster text, extract first link text when present, and extract numeric mass with `kg` unit.
   - Each helper function defensively handles missing elements (returns `None` or a sensible default) and strips reference markers or unicode artifacts (e.g., `♺`, superscripts).

6. Data validation and edge cases

   - Rows are only accepted when they meet a minimum cell-count expectation (e.g., `len(row) >= 9`) and when the row header contains a numeric flight number — this filters out non-data rows and section headers.
   - Missing values are propagated as `None` (converted to NaN in the DataFrame). Numeric parsing (`get_mass`) attempts to extract the first `kg` token; when missing, a zero or NaN can be used depending on downstream needs.

7. Dictionary accumulation and DataFrame construction

   - A `launch_dict` is pre-initialized with keys for each target column. For each valid row, we append cleaned cell values to the corresponding list. After parsing the table(s), the dict is converted to a `pandas.DataFrame` using `pd.DataFrame({k: pd.Series(v) for k,v in launch_dict.items()})` to preserve unequal list lengths safely.

8. Export and reproducibility

   - The resulting DataFrame is exported to CSV (`spacex_web_scraped.csv`) with `index=False`. Commit this CSV or its raw HTML snapshot alongside the notebook for full reproducibility.

9. Practical recommendations for productionizing the pipeline
   - Add logging at each major step (HTTP status, number of rows parsed, number of skipped rows).
   - Add unit tests for helper functions using representative HTML snippets (link-rich cells, missing cells, annotated cells).
   - Consider serializing intermediate artifacts (raw HTML, cleaned row JSON) and adding a checksum of the `oldid` to verify the exact source snapshot used.

This summary should be sufficient for an engineer to reproduce the scrape, extend it to additional snapshots, or adapt it to other wiki tables with similar structure.

## Workflow flowchart

Below is a flowchart describing the end-to-end scraping workflow. First is a Mermaid diagram (rendered if your notebook viewer supports it), followed by an ASCII fallback that is safe to view in any editor.

```mermaid
flowchart TD
    A[Select static snapshot URL (oldid)] --> B[HTTP GET with headers (User-Agent)]
    B --> C[Parse HTML with BeautifulSoup]
    C --> D[Locate target table by class (validate headers)]
    D --> E[Extract & normalize column headers]
    D --> F[For each data row: apply helper extractors]
    F --> G[Append cleaned values to launch_dict lists]
    G --> H[Convert launch_dict to pandas.DataFrame]
    H --> I[Validate, coerce types, parse mass/date/time]
    I --> J[Export cleaned CSV (spacex_web_scraped.csv)]
    J --> K[Optional: save raw HTML snapshot & logs for reproducibility]
```

ASCII fallback:

Select snapshot (oldid) -> HTTP GET (headers/User-Agent) -> Parse HTML (BeautifulSoup)
-> Locate table by class -> Extract headers -> Row-by-row extraction with helper functions
-> Accumulate into launch_dict -> Convert to DataFrame -> Validate & clean -> Export CSV
-> (Optional) save raw HTML and logs for reproducibility.

Notes: a production pipeline should add logging, rate-limiting, retries/backoff, and unit tests for the helper extractors. The mermaid diagram is provided for readers whose toolchain renders it; the ASCII fallback is always visible.


Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [4]:
# use requests.get() method with "static_url" and "headers"ArithmeticError
# assign the response to a object called "response"
response = requests.get(static_url, headers=headers)

Create a `BeautifulSoup` object from the HTML `response`


In [5]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, "html.parser")

Print the page title to verify if the `BeautifulSoup` object was created properly


In [6]:
# Use soup.title attribute
soup.title.string

'List of Falcon 9 and Falcon Heavy launches - Wikipedia'

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [7]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
tables = soup.find_all("table")
# Assign the result to a list called `html_tables`
html_tables = tables

Starting from the third table is our target table contains the actual launch records.


In [8]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [9]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
table_headers = first_launch_table.find_all("th")
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
for th in table_headers:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

Check the extracted column names


In [10]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [11]:
launch_dict = dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict["Date and time ( )"]

# Let's initial the launch_dict with each value to be an empty list
launch_dict["Flight No."] = []
launch_dict["Launch site"] = []
launch_dict["Payload"] = []
launch_dict["Payload mass"] = []
launch_dict["Orbit"] = []
launch_dict["Customer"] = []
launch_dict["Launch outcome"] = []
# Added some new columns
launch_dict["Version Booster"] = []
launch_dict["Booster landing"] = []
launch_dict["Date"] = []
launch_dict["Time"] = []

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [12]:
import re

extracted_row = 0


def _get_first_link_or_text(cell):
    """Return the first <a> text if present, otherwise return stripped cell text."""
    if cell is None:
        return None
    a = cell.find("a")
    if a:
        # Prefer visible text of the link, fallback to get_text
        text = a.string if a.string else a.get_text(strip=True)
        return text.strip() if text else None
    # fallback to full cell text (strip whitespace and references)
    text = cell.get_text(" ", strip=True)
    return text.strip() if text else None


# Extract each table
for table_number, table in enumerate(
    soup.find_all("table", "wikitable plainrowheaders collapsible")
):
    # get table row
    for rows in table.find_all("tr"):
        # check to see if first table heading is as number corresponding to launch a number
        flight_number = None
        flag = False
        if rows.th:
            # get text safely and extract digits (some th contain other tags)
            th_text = rows.th.get_text(" ", strip=True)
            m = re.search(r"\d+", th_text)
            if m:
                flight_number = m.group(0)
                flag = True
        # get table element
        row = rows.find_all("td")
        # if it is number save cells in a dictionary
        if flag and row and len(row) >= 9:
            extracted_row += 1

            # Flight Number value
            launch_dict["Flight No."].append(flight_number)

            datatimelist = date_time(row[0])

            # Date value
            date = (
                datatimelist[0].strip(",")
                if datatimelist and len(datatimelist) > 0
                else None
            )
            launch_dict["Date"].append(date)

            # Time value
            time = (
                datatimelist[1]
                if datatimelist and len(datatimelist) > 1
                else None
            )
            launch_dict["Time"].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = _get_first_link_or_text(row[1])
            if isinstance(bv, str):
                bv = bv.replace("♺", "").strip()
            launch_dict["Version Booster"].append(bv)

            # Launch Site
            launch_site = _get_first_link_or_text(row[2]) or row[2].get_text(
                " ", strip=True
            )
            launch_dict["Launch site"].append(launch_site)

            # Payload
            payload = _get_first_link_or_text(row[3]) or row[3].get_text(
                " ", strip=True
            )
            launch_dict["Payload"].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)

            # Orbit
            orbit = _get_first_link_or_text(row[5]) or row[5].get_text(
                " ", strip=True
            )
            launch_dict["Orbit"].append(orbit)

            # Customer
            customer = _get_first_link_or_text(row[6]) or row[6].get_text(
                " ", strip=True
            )
            launch_dict["Customer"].append(customer)

            # Launch outcome
            launch_outcome = (
                row[7].get_text(" ", strip=True) if row[7] else None
            )
            launch_dict["Launch outcome"].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8]) if len(row) > 8 else None
            launch_dict["Booster landing"].append(booster_landing)

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [13]:
df = pd.DataFrame(
    {key: pd.Series(value) for key, value in launch_dict.items()}
)

In [14]:
df.shape

(124, 11)

In [15]:
df.to_csv("spacex_web_scraped.csv", index=False)

In [16]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success [ 9 ],F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success [ 20 ],F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
